#  Age range estimation based on CNN DenseNet201
- This project represents my undergraduate final work for the Federal Technological University of Parana - Brazil in 2019, whose objective was to realize age range estimation based on a convolutional neural network.

- With the technological evolution of social networks and online communities, privacy and security on the Internet have become essential. The high number of information shared by the network supports the spread of illicit content involving child pornography. Computer vision, based on neural networks as a deep learning technique, can recognize characteristics associated with the classification of minors and adults.

-  The main focus was to provide subsidies for tools that can use deep learning to identify minors in eventual child pornography content.

- For more details about the objectives and technical specifications, read the article in Portuguese by the `ARTIGO.pdf` file. 

### Dataset
![enter image description here](https://susanqq.github.io/UTKFace/icon/logoWall2.jpg)
- As data, the recognition process occurred with cropped face images. 

- The dataset used to train the neural network was __UTKFace Dataset__, made for research based on age, gender, and race.

- Files format: ``[age]_[gender]_[race]_[date&time].jpg``
	- `[age]:` integer from 0 to 116, indicating the age;
	- `[gender]:` 0 (male) or 1 (female);
	- `[race]:` integer from 0 to 4, denoting White, Black, Asian, Indian, and Others (like Hispanic, Latino, Middle Eastern);
	- `[date&time]:` format of yyyymmddHHMMSSFFF, showing the date and time an image was collected to UTKFace.
- As the dataset is very large, I provide some sample files in `img` folder.

- The UTKFace Dataset can be downloaded [here](https://susanqq.github.io/UTKFace/).

### CNN model
- To perform the age estimation process, I choose as a deep learning model the classification segment state of art named DenseNet, according to ImageNet dataset report errors (21.46 in top-1 and 5.54 in top-5).
- Below, a dense layer structure example:
![dense layer](./img/DenseNet.png)



# Mount Drive in Google Colab and Libraries

In [1]:
from google.colab import drive

GOOGLE_DRIVE_MOUNT_DIR = "/content/drive"
drive.mount(GOOGLE_DRIVE_MOUNT_DIR, force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#checking gpu available
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import tensorflow as tf
import numpy as np
import keras.backend as K
import datetime, os
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.applications.densenet import DenseNet201, preprocess_input
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras import losses, optimizers, activations
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report

Using TensorFlow backend.


# Ages classes

In [3]:
#dataset with 2 classes
TRAIN_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/train"
VAL_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/val"
TEST_PATH = "../content/drive/My Drive/Rede Neural/MY DATASETS/SPLIT Binary Ages Dataset/test"

In [ ]:
#dataset with 4 classes
TRAIN_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/BALANCED UTK 4 Classes/train"
VAL_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/BALANCED UTK 4 Classes/val"
TEST_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/BALANCED UTK 4 Classes/test"

In [ ]:
#dataset with 7 classes
TRAIN_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/FLIP Split Dataset Ages Balanced/train"
VAL_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/FLIP Split Dataset Ages Balanced/val"
TEST_PATH = "/content/drive/My Drive/Rede Neural/MY DATASETS/test - balanced"

# Model Customization

- The DenseNet201 base model was imported with ImageNet weights and the same UTK Face Dataset resolution (200x200).
- I used some fine-tuning techniques to improve the classification performance:
	- **GlobalAveragePooling2d**: This layer applied after the base model helps the network to learn the correlation between all the filters during the training process. More specifically, a GlobalAveragePooling2d feed the filter average value through the next layers.
	- **Dropout**: As a regularization method, the Dropout layer keeps a percentage of neurons turned off during the training process (chosen randomly at each epoch). This process avoids co-dependencies between parameters that can lead neurons to overfitting. In this project, with 50% of neurons turned off, I obtained the best classification performance.
- Loss and Activation Functions: 
    - **Categorical Cross-entropy**: A loss function calculates the gradients responsible for updating the neural network weights in the training process. The categorical cross-entropy is a loss function applied to multi-class classification tasks that calculates the probabilities of each class. Thereby, the categorization occurs based on the highest probability value.
    - **Softmax**: Activation functions are mathematical equations responsible for the output value of a neural network. Softmax is an activation function highly recommended when the loss function is the categorical cross-entropy. This because the output values is a probabilities vector between 0 and 1. So, the softmax function can compare probabilities distributions resizing the model output to the right properties.
    

In [ ]:
#change number of classes according your configuration
CLASSES = 2

base_model = DenseNet201(weights='imagenet', include_top=False, input_shape = (200,200,3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# training all the layers
for layer in base_model.layers:
  layer.trainable = True

filepath = "/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/2classes.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]
      
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

74842112/74836368 [==============================] - 1s 0us/step


In [ ]:
#visualize trainable layers
for i, layer in enumerate(model.layers):
   print(i, layer.name, layer.trainable)

In [ ]:
model.summary()

# Data preparation
- This code cell below gets all images for train, validation, and test from the Google Drive dataset directory.

In [ ]:
WIDTH = 200
HEIGHT = 200
BATCH_SIZE = 32


# data normalization
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

#obtaining data from Google Drive directory
train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(HEIGHT, WIDTH),
		batch_size=BATCH_SIZE,
		class_mode='categorical',
    shuffle=True)
    
validation_generator = val_datagen.flow_from_directory(
    VAL_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

train_generator.class_indices


Found 7196 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 254 images belonging to 2 classes.


{'1-17': 0, '18-r': 1}

# Initializing Training

In [ ]:
EPOCHS = 20
BATCH_SIZE = 32


history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_list)

# Plot History of training

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Confusion Matrix and Classification Report

In [ ]:
Y_pred = model.predict(test_generator, batch_size=BATCH_SIZE, verbose=1, steps=len(test_generator), workers=0)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))
print('Classification Report')
target_names = ['1-17', '18-116']
print(classification_report(test_generator.classes, y_pred, digits=4, target_names=target_names))

# Reload Model and Training process

- If necessary, the model trained can be reloaded and retrained. 

In [ ]:
model2 = load_model('/content/drive/My Drive/Rede Neural/Checkpoints/DenseNet-weights.best-class-newdivision-large.hdf5')

for layer in model2.layers:
  layer.trainable = True

model2.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model2.summary()

Reset generators and execute the Data Preparation process to create another random seed to the images.

In [ ]:
train_generator.reset()
validation_generator.reset()

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32

filepath = "/content/drive/My Drive/Rede Neural/Checkpoints/DenseNet-weights.best-class.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks = [checkpoint]


history = model2.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_list)
  
NAME = "model-DenseNet-4-classes"  
model2.save(f"/content/drive/My Drive/Rede Neural/Models/Ages Classes/{NAME}.model")  

# Making Predictions

- In this code section, the best model trained can be loaded to generate the predictions in a CSV file.  

*For another prediction in the same process, reset test_generator to get a new random seed to images.

In [ ]:
test_generator.reset()

In [4]:
HEIGHT = 200
WIDTH = 200
BATCH_SIZE = 32

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

test_generator.class_indices

Found 254 images belonging to 2 classes.


{'1-17': 0, '18-r': 1}

In [6]:
model = load_model('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-2classes.hdf5')

pred = model.predict(test_generator, batch_size=32, verbose=1, steps=len(test_generator), workers=0)
predicted_class_indices = np.argmax(pred, axis=1)
labels = (test_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

filenames = test_generator.filenames
results = pd.DataFrame({"Filename":filenames,
                        "Predictions":predictions})

#save CSV file
results.to_csv(r'/content/drive/My Drive/Rede Neural/TESTS/2classes_test.csv')

8/8 [==============================] - 2s 277ms/step


In [7]:
#showing the filenames and respective predictions
results

,Filename,Predictions
0,1-17/10_0_0_20170110220654150.jpg.chip.jpg,1-17
1,1-17/10_1_0_20170109203501969.jpg.chip.jpg,1-17
2,1-17/10_1_2_20170109201545634.jpg.chip.jpg,1-17
3,1-17/10_1_3_20170109203848078.jpg.chip.jpg,1-17
4,1-17/10_1_4_20161223225900460.jpg.chip.jpg,1-17
...,...,...
249,18-r/84_1_2_20170120225608458.jpg.chip.jpg,18-r
250,18-r/85_0_0_20170117192351268.jpg.chip.jpg,18-r
251,18-r/86_1_0_20170110180113129.jpg.chip.jpg,18-r
252,18-r/87_1_0_20170120225808097.jpg.chip.jpg,18-r


# Reload best checkpoint and Confusion Matrix

- This code cell below loads the best model saved with Checkpoint supervision and returns the Confusion Matrix and Classification Report with precision, recall, and f1-score values.

In [ ]:
def show_results(model_path, test_generator, t_names):
  model = load_model(model_path)

  Y_pred = model.predict(test_generator, batch_size=32, verbose=1, steps=len(test_generator), workers=0)
  y_pred = np.argmax(Y_pred, axis=1)
  print('Confusion Matrix:')
  print(confusion_matrix(test_generator.classes, y_pred))
  print('\nClassification Report:')
  target_names = t_names
  print(classification_report(test_generator.classes, y_pred, digits=4, target_names=target_names))


# Experiments

- I saved the best models for each experiment and reloaded them to their respective cells below to show the results.

## Exp 1: classification in 7 age ranges: 
## ‘1–10’, ‘11–15’, ‘16–18’, ‘19–25’, ‘26–40’, ‘41–60’, ‘61–116’

- Objective: There is a lot of machine learning projects that had explored CNN capacity to realize age recognition based on facial images, with some of them splitting the ages in groups. This experiment had the goal of split the ages in a similar number of groups from some reference authors for the entire project.
- Distribution of split dataset:

| Dataset | Number of images |
| -- | -- |
| Train | 4.998 |
| Validation | 1.400 |
| Test | 350 |
| **Total** | **6.748** | 

In [ ]:
show_results('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-7classes.hdf5', 
             test_generator, 
             ['1-10', '11-15', '16-18', '19-25', '26-40', '41-60', '61-116'])

11/11 [==============================] - 108s 10s/step
Confusion Matrix:
[[41  5  2  0  2  0  0]
 [14 15 14  5  1  1  0]
 [ 2  5 23 15  3  1  1]
 [ 0  0  6 26 16  2  0]
 [ 0  0  0  3 41  5  1]
 [ 0  0  1  1 11 24 13]
 [ 0  0  0  0  0  3 47]]

Classification Report:
              precision    recall  f1-score   support

        1-10     0.7193    0.8200    0.7664        50
       11-15     0.6000    0.3000    0.4000        50
       16-18     0.5000    0.4600    0.4792        50
       19-25     0.5200    0.5200    0.5200        50
       26-40     0.5541    0.8200    0.6613        50
       41-60     0.6667    0.4800    0.5581        50
      61-116     0.7581    0.9400    0.8393        50

    accuracy                         0.6200       350
   macro avg     0.6169    0.6200    0.6035       350
weighted avg     0.6169    0.6200    0.6035       350



## Exp 2: classification in 4 age ranges: ‘1–13’, ‘14–17’, ‘18–35’, ‘36–116’

- Objective: According to Brazilian law, possible crimes against minors under the age of 13 have more severe sentences. Thereby, this experiment had the objective of testing the CNN performance to differentiate two age groups of minors: 1-13 and 14-17. Besides these two groups, the 18-35 and 36-116 adult classes.  
- Distribution of split dataset:

| Dataset | Number of images |
| -- | -- |
| Train | 3.519 |
| Validation | 800 |
| Test | 400 |
| **Total** | **4.719** |

In [ ]:
show_results('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-4classes.hdf5',
             test_generator,
             ['1-13', '14-17', '18-35', '36-116'])

13/13 [==============================] - 4s 300ms/step
Confusion Matrix:
[[84 12  4  0]
 [20 72  8  0]
 [ 8 23 57 12]
 [ 3  4 14 79]]

Classification Report:
              precision    recall  f1-score   support

        1-13     0.7304    0.8400    0.7814       100
       14-17     0.6486    0.7200    0.6825       100
       18-35     0.6867    0.5700    0.6230       100
      36-116     0.8681    0.7900    0.8272       100

    accuracy                         0.7300       400
   macro avg     0.7335    0.7300    0.7285       400
weighted avg     0.7335    0.7300    0.7285       400



## Exp 3: classification in 2 age ranges: ‘1–17’, ‘18–116’

- Objective: As the main objective of this project, to differentiate between minors and adults.
- Distribution of split dataset:

| Dataset | Number of images |
| -- | -- |
| Train | 7.196 |
| Validation | 1.000 |
| Test | 254 |
| **Total** | **8.450** |

In [ ]:
show_results('/content/drive/My Drive/Rede Neural/Checkpoints/Age Classification/DenseNet-2classes.hdf5',
             test_generator,
             ['1-17', '18-116'])

8/8 [==============================] - 2s 309ms/step
Confusion Matrix:
[[122   5]
 [  3 124]]

Classification Report:
              precision    recall  f1-score   support

        1-17     0.9760    0.9606    0.9683       127
      18-116     0.9612    0.9764    0.9688       127

    accuracy                         0.9685       254
   macro avg     0.9686    0.9685    0.9685       254
weighted avg     0.9686    0.9685    0.9685       254



# Results
- Were tested many configurations to find the best accuracy. The table below shows the most suitable ones.

|Hyperparameter| Value |
|--|--|
| Loss Function | Categorical Crossentropy |
| Dropout Keep Probability | 0.5 |
| Batch Size | 32 |
| Learning Rate | 0.001(Exp 1) 0.0001(Exp 2 and 3) |

- For the proposed experiments, were used accuracy and f1-score as metrics, with the results below: 

|Experiment| Accuracy | f1-score
|--|--|--|
| 1 | 62.00% | 60.35% |
| 2 | 73.00% | 72.85% |
| 3 | 96.85% | 96.85% |

# Conclusions
- The main focus was to verify the deep learning capacity to differentiate between minors and adults. With almost 97% accuracy, the DenseNet architecture was highly performative.
- In terms of applicability, this work showed that convolutional neural models can be seriously considered as an accurate mechanism for the identification of possible minors in digital images, considering different intervals of age group as a target for classification.